In [73]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [74]:
txt = 'great product, I love it!!!!!!'

In [75]:
def remove_punctuation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')

    return text

In [76]:
# Model loading
with open('../static/model/sentiment_model.pkl', 'rb') as file:
    model = pickle.load(file)

In [77]:
with open ('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [78]:
# Importing vocabulary
vocab = pd.read_csv('../static/model/corpora/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [79]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [80]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['text'])
    data['tweet'] = data['text']

    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data['tweet'] = data['tweet'].apply(remove_punctuation)
    data['tweet'] = data['tweet'].str.replace('\d+', ' ', regex=True)
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    
    return data['tweet']

<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Asus\AppData\Local\Temp\ipykernel_10248\2550991882.py:8: SyntaxWarning: invalid escape sequence '\d'
  data['tweet'] = data['tweet'].str.replace('\d+', ' ', regex=True)


In [81]:
preprocessed_txt = preprocessing(txt)
preprocessed_txt

0    great product love
Name: tweet, dtype: object

In [82]:
def vectorizer(ds, vocabulary):
    vectorized_list = []

    for sentence in ds:
        sentence_list = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_list[i] = 1
        vectorized_list.append(sentence_list)
    vectorized_list_new = np.asarray(vectorized_list, dtype=np.float32)
    
    return vectorized_list_new

In [83]:
vectorized_txt = vectorizer(preprocessed_txt, tokens)
vectorized_txt

array([[0., 0., 0., ..., 0., 0., 0.]], shape=(1, 1154), dtype=float32)

In [84]:
def get_prediction(vectorized_txt):
    prediction = model.predict(vectorized_txt)
    if prediction == 0:
        return 'positive'
    else:
        return 'negetive'


In [85]:
txt = 'I love it'
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
print('Sentiment: ', prediction)

Sentiment:  positive
